![](injury.jpeg)


Contents

1. [Introduction](#1-introduction)  
1.1 [Background](#11-background)  
1.2 [Objectives](#12-objectives)
2. [Data Preparation](#2-data-preparation)  
2.1 [Imports](#21-imports)  
2.2 [Google Cloud Storage](#22-GCS)  
2.3 [Dataframe Construction & Data Cleaning](#23-dfc&dc)  
2.4 [Feature Correlations](#24-ft_corr)
3. [Model Development](#3-MD).  
3.1[Training the Model](#31-TM)  
3.2[Plotting the Model](#32-PM)  
4. [Model Testing](#4-MT)  
4.1 [Plotting the Test Results](#4.1-PTR)  
5. [Outlier Handling](#5-OH)
6. [Conclusion](#6-Conc)
        

##  Introduction

### Background
<a id="11-background"><a>

The National Basketball Association (NBA) is one of the most popular and competitive professional sports leagues globally, captivating millions of fans with its fast-paced action and exceptional athleticism. However, along with the excitement and intensity of the game, player injuries have become a significant concern for teams, players, and fans alike.

Understanding and predicting the number of injuries that occur in the NBA year over year has become a crucial area of research and analysis. The impact of injuries goes beyond individual players, as it can significantly influence team performance, game strategies, and overall league dynamics. Building a predictive model for NBA injuries aims to gain valuable insights into injury patterns, risk factors, and the potential impact on team dynamics.

The importance of such a predictive model lies in its ability to assist various stakeholders in the NBA ecosystem. Team management can use the model to proactively manage player workload, optimize training and recovery protocols, and minimize injury risks. Coaches can adjust game strategies and rotations based on injury predictions, aiming to maximize player availability and team performance. Additionally, players themselves can benefit from injury forecasting by adopting personalized training programs and injury prevention measures.

Furthermore, the predictive model contributes to the broader field of sports science and sports medicine. By analyzing historical injury data and identifying factors that contribute to injuries, valuable insights can be gained into injury prevention, rehabilitation techniques, and overall player health and well-being.

By leveraging advanced data analysis and machine learning techniques, a robust predictive model for NBA injuries has been developed. This model considers various factors such as player workload, playing style, previous injury history, game schedule, and other relevant variables to forecast the likelihood of injuries occurring throughout the season. The model's predictions provide valuable information for teams, coaches, and players, enabling them to make data-driven decisions to enhance player safety, optimize performance, and ultimately elevate the overall quality of the NBA game.

Through this research, efforts have been made to contribute to the ongoing initiatives of the NBA and the sports community as a whole in promoting player health, reducing injury rates, and fostering a more sustainable and competitive environment.

### Objectives
<a id="12-objectives"></a>

The objectives of the NBA injury predictive model are twofold. Firstly, the model aims to accurately predict the total number of injuries that will occur in the NBA during a calendar season. By training the model on historical injury data and leveraging advanced machine learning techniques, it strives to provide reliable forecasts that can assist teams, coaches, and players in proactive injury management and planning.

Secondly, the model seeks to gain insights into the factors that have the greatest impact on the overall number of injuries in the NBA. These insights can contribute to a better understanding of injury risk factors and inform strategies for injury prevention and player safety.



## Data Preparation
<a id="2-data-preparation"></a>

### Imports
<a id="21-imports"></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score 
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.ticker as ticker
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri


### Google Cloud Storage
<a id="22-GCS"><a>

In [2]:
from google.cloud import storage
import io
key_path = "/Users/antho/Documents/GCP/nba-injury-forecasting-59f4078886ad.json"
client = storage.Client.from_service_account_json(key_path)

In [3]:
bucket_name = 'nba-data-2023-v2'
bucket = client.get_bucket(bucket_name)

In [4]:
injData = 'Injuries.csv'
blob = bucket.blob(injData)
blob.download_to_filename(injData)
injuryData = pd.read_csv(injData)

In [5]:
# Download and store the data from AverageTravel.csv, AveragePlayerAge.csv, and NBASalaries.csv
blob_travel = bucket.blob('AverageTravel.csv')
content_travel = blob_travel.download_as_text()
annual_travel = pd.read_csv(io.StringIO(content_travel))

blob_age = bucket.blob('AveragePlayerAge.csv')
content_age = blob_age.download_as_text()
average_age = pd.read_csv(io.StringIO(content_age))

blob_salary = bucket.blob('NBASalaries.csv')
content_salary = blob_salary.download_as_text()
average_salary = pd.read_csv(io.StringIO(content_salary))

### Dataframe Construction & Data Cleaning
<a id="23-dfc&dc"></a>

In [6]:
injuryData.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)
injuryData.rename(columns={'Notes': 'Injury Type'}, inplace=True)
injuryData.rename(columns={'Relinquished': 'Player'}, inplace=True)

In [7]:
# Setting the objects in the column to a pandas datetime object
injuryData['Date'] = pd.to_datetime(injuryData['Date'])

#Grouping rows by 'decade' column
injuryData['Year'] = (injuryData['Date'].dt.year)

>__Let's now remove all rows where the players have been acquired back from the IL__


In [8]:
# Removing the Acquired column removes all duplicates 
injuryData = injuryData[injuryData['Acquired'].isna()]
del injuryData['Acquired']

> __Now let's calculate the total sum of injuries for each Year__

 > ** *It is important to note that the NBA did not have and IR/IL until the 1989-1990 season. Prior to this injured players were still members of the active roster. For this reason, it's likely many injuries do not appear in the dataset.*

In [ ]:
# We only want to include data from the years after 1989 
yearly_injuries = {}
total_injuries = 0
for year, group in injuryData.groupby('Year'):
    if year > 1989: 
        injuries = group.shape[0]
        yearly_injuries[year] = injuries
        total_injuries += injuries

> **Prior to the 2015 season, teams in the NBA were only allowed 12 men on their active roster**

In [ ]:
# Adding total number of players in the League per year
df = pd.DataFrame(yearly_injuries.items(), columns=['Year', 'Injuries'])
df['Players'] = np.where(df['Year'].astype(int) <= 2015, 354, 450)
#Turning the total number of players feature into binary values 
df['Players'] = df['Players'].apply(lambda x: 0 if x == 354 else 1)
df['isleagueExpansion'] = np.where(df['Year']<=2004, 0, 1)
df['Players'] = df['Players'].astype(int)

In [ ]:
# adding all the data from outside data sources to the Dataframe
df = pd.concat([df, annual_travel['Miles Traveled']], axis=1)
df = pd.concat([df, average_age['Average Age']], axis = 1)
df = pd.concat([df, average_salary['salary']], axis = 1)

In [ ]:
# adding the total number of games played in the NBA each year to the Dataframe
def get_games_played(df):

    games_played = {
      year: 6620
      for year in range(1990, 2024)
  }

  # the following seasons were shortened due to either lockout(1999, 2011) or COVID-19(2020-2021)
    games_played[1999] = 4100
    games_played[2011] = 4096
    games_played[2020] = 4384
    games_played[2021] = 4384
    df['Games Played'] = df.Year.map(games_played)
    df['Games Played'] = df['Games Played'].astype(int)

get_games_played(df)

> **A feature needs to be added to account for the shortened seasons due to lockouts and the COVID-19 virus**

In [ ]:
# add feature for injuries in the previous season
df['prevYrINJ'] = df['Injuries'].shift(1)
df['prevYrINJ'].fillna(0, inplace=True)
df['prevYrINJ'] = df['prevYrINJ'].astype(int)
df.loc[0, 'prevYrINJ'] = 64

### Feature Scaling

In [ ]:
# Adding feature to account for a shortened season(lockouts and COVID-19 years)
df['Shortened Season'] = np.where(df['Year'].isin([1999, 2011, 2020, 2021]), 1, 0)
df

>  **To prevent larger values from skewing the model I will be using max-min normalization to set all feature values between 0 and 1**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def min_max_scaling(df):
    # copy the dataframe
    df_norm = df.copy()
    scaling_params = {}
    # apply min-max scaling
    for column in df_norm.columns:
        if column != 'Year':
            # Store scaling parameters for each column
            min_val = df_norm[column].min()
            max_val = df_norm[column].max()
            scaling_params[column] = {'min_val': min_val, 'max_val': max_val}
            df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        
    return df_norm, scaling_params
    
# call the min_max_scaling function
df, scaling_params = min_max_scaling(df)

# df_scaled contains the min-max scaled dataframe


### Feature Correlations
<a id="24-ft_corr"></a>

In [ ]:
# calculating the correlation for the two features I will use to train my model
features = ['Players', 'Games Played', 'Miles Traveled', 'Shortened Season', 'Average Age', 'salary', 'prevYrINJ']

for feature in features:
    correlation = df[feature].corr(df['Injuries'])
    print(f"{feature} feature: {correlation}")


## Model Development

### Training the Model

In [ ]:
df.head()

In [ ]:
#Using Linear Regression function to fit our model
train_data = df[:26]

X_train = np.array(train_data[['Players', 'Games Played', 'Miles Traveled', 'Shortened Season', 'Average Age', 'salary', 'prevYrINJ']])
y_train = train_data['Injuries']
reg = LinearRegression()


fit_model = reg.fit(X_train, y_train)
print(df.shape)
X_train.shape


In [ ]:
# using a scikit-learn attribute, the coef represents the wieghts assigned to different features 
print(f"The Weight of feature(s): {fit_model.coef_}")

In [ ]:
# getting the intercept(bias) for our fitted model
print(f"The Bias of the model: {fit_model.intercept_}")

In [ ]:
# Adding our insample prediction for each year to our training data set
train_data.loc[:, 'prediction_insample'] = fit_model.predict(X_train).copy()
train_data.head()

In [ ]:
# Calculating the error, MSE, and ABS error for each year
train_data['error'] = train_data.prediction_insample - train_data.Injuries
train_data['squared_error'] = train_data['error']**2
train_data['abs'] = abs(train_data['error'])

In [ ]:
train_data[['error', 'squared_error', 'abs']].mean()

In [ ]:
# Converting the scaled predictions back to the original scale
def value_inverse(df):
    for column in df.columns:
        if column != 'Year':
            if column in scaling_params:
                min_val = scaling_params[column]['min_val']
                max_val = scaling_params[column]['max_val']
                df[column] = (df[column] * (max_val - min_val)) + min_val
            elif column == 'prediction_insample': 
                # If 'prediction_insample' column, use the scaling parameters of 'Injuries' column
                min_val = scaling_params['Injuries']['min_val']
                max_val = scaling_params['Injuries']['max_val']
                df[column] = (df[column] * (max_val - min_val)) + min_val
    return df
train_data = value_inverse(train_data)


### Plotting the Model

In [ ]:
sns.set(style="darkgrid")
sns.lineplot(x=train_data['Year'], y=train_data['Injuries'], data=train_data)
sns.lineplot(x=train_data['Year'], y=train_data['prediction_insample'], data=train_data)
plt.xlabel('Year')
plt.ylabel('Number of Injuries')
plt.title('Number of Injuries Year by Year')
actual_injuries_legend = plt.Line2D([], [], color='blue', label='Actual Injuries')
predicted_injuries_legend = plt.Line2D([], [], color='orange', label='Predicted Injuries')
plt.legend(handles=[actual_injuries_legend, predicted_injuries_legend])



## Model Testing
<a id="4-MT"><a>

In [ ]:
test_data = df[26:]
 
X_test = np.array(test_data[['Players', 'Games Played', 'Miles Traveled', 'Shortened Season', 'Average Age', 'salary', 'prevYrINJ']])
y_test = test_data['Injuries']


In [ ]:
y_pred = fit_model.predict(X_test)
# Adding our insample prediction for each year to our test data set
test_data.loc[:, 'prediction_insample'] = y_pred.copy()
test_data.head()

In [ ]:
test_data['error'] = test_data.prediction_insample - test_data.Injuries
test_data['squared_error'] = test_data['error']**2
test_data['abs'] = abs(test_data['error'])


In [ ]:
df.describe()["Injuries"]

### Plotting the Test Results
<a id="4.1-PTR"><a>

In [ ]:
# first let's convert the columns back to their original values 
test_data = value_inverse(test_data)


In [ ]:
sns.set(style="darkgrid")
sns.lineplot(x=test_data['Year'], y=test_data['Injuries'], data=test_data)
sns.lineplot(x=test_data['Year'], y=test_data['prediction_insample'], data=test_data)
plt.legend()
plt.xlabel('Year')
plt.ylabel('Number of Injuries')
plt.title('Number of Injuries Year by Year')
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
plt.show()


> **Outliers appeared in years 2020 and 2021 for the data**  
**To combat this, I will be using Huber Regression**

## Outlier Handling
<a id="5-OH"><a>

In [ ]:
# I currently have outlier for my 2020 and 2021 predictions
# I will attempt to use Huber Regression to account for these outliers 
from sklearn.linear_model import HuberRegressor
huber_dataset = df[26:]
X_huber = huber_dataset[[ 'Games Played', 'Shortened Season', 'Miles Traveled', 'Average Age', 'salary', 'prevYrINJ']]
y_huber = huber_dataset['Injuries']

huber_reg = HuberRegressor()
fitted_model = huber_reg.fit(X_huber, y_huber)

y_pred = fitted_model.predict(X_huber)

huber_dataset['prediction_insample'] = y_pred


In [ ]:
huber_dataset['error'] = huber_dataset.prediction_insample - huber_dataset.Injuries
huber_dataset['squared_error'] = huber_dataset['error']**2
huber_dataset['abs'] = abs(huber_dataset['error'])

In [ ]:
# Once again we must revert the values back to there original range: 
huber_dataset = value_inverse(huber_dataset)

In [ ]:
plt.plot(huber_dataset['Year'], huber_dataset['prediction_insample'], 'r-', label='Predicted Data')
sns.set(style="darkgrid")
sns.lineplot(x='Year', y='Injuries', data=huber_dataset)
plt.legend()
plt.xlabel('Year')
plt.ylabel('Number of Injuries')
plt.title('Number of Injuries Year by Year')
ax = plt.gca()
ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True))
plt.show()

In [ ]:
# give a run down of the features and the coeffecients
# (for every thousand mile travelled injuries increased by 2 percent)

In [ ]:
weights = fitted_model.coef_
weights

## Conclusion
<a id="6-Conc"><a> 

> **Below is a graph representing the features sorted from most to least important**

In [ ]:
sort = weights.argsort()
plt.barh(X_huber.columns[sort], weights[sort])
plt.xlabel("Feature Importance")
plt.show()

### Summary 

Not Surprisingly the feature that had the largest impact on the total number of injuries was the total miles traveled. As the NBA has expanded and become a global superpower, so to has the demands on athletes grown in terms of their total travel. The next greatest factors were salary and average age. Again not much of a surprise. As the league has grown both in popularity the league has garnered massive revenue. This has allowed for more time and effort to be placed into developing there training staff along with their athletes. Average Age as well, logically holds up. The older the athlete, the more their bodies tend to break down.  

The Surprising statistic was to see a negative feature importance for Games Played. It appear as if the weight of the Games Played feature actually skews in the opposite direction. According to the model, the more games that are played, the lower the likelihood of injury, across the league. However, this may not be entirely accurate. It's worth noting that the 2019-2020 and 2020-2021 COVID years could be greatly impacting this value. As those seasons saw both a shortened number of games, while also displaying anomalous values for injuries as many players were sidelined with the virus. 